In [1]:
import pandas as pd
from llama_index import download_loader
import re
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from llama_index.embeddings import OpenAIEmbedding
from os import environ
import streamlit as st

In [2]:
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

In [3]:
# Load the markdown reader from the hub
MarkdownReader = download_loader("MarkdownReader")
markdownreader = MarkdownReader()

In [4]:
docs = []
df_pre = pd.read_csv("./data/uscis.csv").dropna()
for row in range(len(df_pre)):
    row_data = df_pre.iloc[row]
    heading = row_data.heading
    link = row_data.link
    text = row_data.text
    new_text = cleanhtml(text)
    new_docs = markdownreader.load_data(file=None, content=new_text, extra_info={
        "page": heading,
        "link": link,
    })
    docs.extend(new_docs)

In [5]:
environ["OPENAI_API_KEY"] = st.secrets["OPEN_AI_API_KEY"]
environ["TOKENIZERS_PARALLELISM"] = "false"
BATCH_SIZE = 128

In [6]:
embed_model = OpenAIEmbedding(embed_batch_size=BATCH_SIZE)
service_context = ServiceContext.from_defaults(embed_model=embed_model)

In [7]:
index = VectorStoreIndex.from_documents(
    documents=docs, 
    service_context=service_context,
    show_progress=True
)

Parsing nodes:   0%|          | 0/3538 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4188 [00:00<?, ?it/s]

In [8]:
index.storage_context.persist("ask_priya_index")